In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
from dotenv import load_dotenv

load_dotenv()

False

In [5]:
KEY ="enter your key here"

In [6]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo")

/var/folders/vn/t_h6pnkj18v28kq_7dvxyyw80000gn/T/ipykernel_3885/746436948.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo")


In [ ]:
llm

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1":{
        "mcq" : "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq" : "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq" : "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "4":{
        "mcq" : "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "5":{
        "mcq" : "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    }
}

In [10]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to\
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}


"""

In [11]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [12]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose="True")

/var/folders/vn/t_h6pnkj18v28kq_7dvxyyw80000gn/T/ipykernel_3885/1817962920.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose="True")


In [13]:
TEMPLATE2 = """
Text:{text}
You are an expert English grammarian and writer. Given a multiple choice quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for the complexity,
if the quiz is not as per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students ability.
Quiz_MCQs:
{quiz}


Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE2)

In [16]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review", verbose="True")

##### I have created a quiz chain and then the review chain now I'm going to create a squentail chain to join both of them

In [17]:
# create an object of this sequential chain
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone", "response_json"],
    output_variables=["quiz","review"],
    verbose="True",
)

### I'm going to read the data from data.txt from particular file

In [22]:
file_path=r"/Users/aqsashakoor/Mydocuments/openAI/MCQGen/data.txt"

In [19]:
file_path

'/Users/aqsashakoor/Mydocuments/openAI/MCQGen/experminet/data.txt'

In [23]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [24]:
print(TEXT)

Sayyid Ibrahim Husayn Shadhili Qutb[a] (9 October 1906 – 29 August 1966) was an Egyptian political theorist and revolutionary who was a leading member of the Muslim Brotherhood. He is dubbed the "father of Salafi jihadism", the religio-political doctrine that underpins the ideological roots of global jihadist organisations such as al-Qaeda and ISIL.

Author of 24 books,[5] with around 30 books unpublished for different reasons (mainly destruction by the state),[6] and at least 581 articles,[7] including novels, literary arts critique and works on education, he is best known in the Muslim world for his work on what he believed to be the social and political role of Islam, particularly in his books Social Justice and Ma'alim fi al-Tariq (Milestones). His magnum opus, Fi Zilal al-Qur'an (In the Shade of the Qur'an), is a 30-volume commentary on the Quran.[8]

During most of his life, Qutb's inner circle mainly consisted of influential politicians, intellectuals, poets and literary figures

In [25]:
# serialize the python dictionary into a JSON-formatted string
json_str = json.dumps(RESPONSE_JSON)


In [26]:
NUMBER=5
SUBJECT="History"
TONE="Formal"

In [27]:
#https://python.langchain.com/v0.1/docs/modules/model_io/llms/token_usage_tracking/

# how to setup token usage Tracking in Langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text" : TEXT,
            "number" : NUMBER,
            "subject" : SUBJECT,
            "tone" : TONE,
            "response_json" : json_str
        }
    )

/var/folders/vn/t_h6pnkj18v28kq_7dvxyyw80000gn/T/ipykernel_3885/3951843590.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Sayyid Ibrahim Husayn Shadhili Qutb[a] (9 October 1906 – 29 August 1966) was an Egyptian political theorist and revolutionary who was a leading member of the Muslim Brotherhood. He is dubbed the "father of Salafi jihadism", the religio-political doctrine that underpins the ideological roots of global jihadist organisations such as al-Qaeda and ISIL.

Author of 24 books,[5] with around 30 books unpublished for different reasons (mainly destruction by the state),[6] and at least 581 articles,[7] including novels, literary arts critique and works on education, he is best known in the Muslim world for his work on what he believed to be the social and political role of Islam, particularly in his books Social Justice and Ma'alim fi al-Tariq (Milestones). His magnum opus, Fi Zilal al-Qur'an (In the Shade of the Qur'an), is a 30-volume commentary on the Quran.[8]

During most of his life

In [28]:
#let's show the number of tokens , input tokens, output tokens and total cost
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")


Total Tokens: 6730
Prompt Tokens: 6258
Completion Tokens: 472
Total Cost (USD): $0.010331


In [29]:
 #let try to get a response or quiz
response

{'text': 'Sayyid Ibrahim Husayn Shadhili Qutb[a] (9 October 1906 – 29 August 1966) was an Egyptian political theorist and revolutionary who was a leading member of the Muslim Brotherhood. He is dubbed the "father of Salafi jihadism", the religio-political doctrine that underpins the ideological roots of global jihadist organisations such as al-Qaeda and ISIL.\n\nAuthor of 24 books,[5] with around 30 books unpublished for different reasons (mainly destruction by the state),[6] and at least 581 articles,[7] including novels, literary arts critique and works on education, he is best known in the Muslim world for his work on what he believed to be the social and political role of Islam, particularly in his books Social Justice and Ma\'alim fi al-Tariq (Milestones). His magnum opus, Fi Zilal al-Qur\'an (In the Shade of the Qur\'an), is a 30-volume commentary on the Quran.[8]\n\nDuring most of his life, Qutb\'s inner circle mainly consisted of influential politicians, intellectuals, poets an

In [30]:
quiz=response.get("quiz")

In [31]:
quiz=json.loads(quiz)

In [33]:
# data frame for this dictionary
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | " .join(
        [
        f"{option}:{option_value}"
        for option, option_value in value["options"].items()
    ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq ,"Choices" : options, "Correct": correct})

In [110]:
quiz_table_data

[{'MCQ': "What was the main reason for Sayyid Qutb's criticism of American culture?",
  'Choices': 'a:Their support for the new Israeli state | b:Their love of classical music | c:Their restrictions on divorce | d:Their materialism',
  'Correct': 'd'},
 {'MCQ': 'Where did Sayyid Qutb study the educational system of the United States?',
  'Choices': "a:University of Northern Colorado | b:Stanford University | c:Both a and b | d:Wilson Teachers' College",
  'Correct': 'c'},
 {'MCQ': 'What was the title of the first major theoretical work of religious social criticism by Sayyid Qutb?',
  'Choices': 'a:Social Justice in Islam | b:The Mission of the Poet in Life | c:The America that I Have Seen | d:Ashwak (Thorns)',
  'Correct': 'a'},
 {'MCQ': "What did Sayyid Qutb find primitive and 'shocking' about American life?",
  'Choices': 'a:Their artistic taste | b:Their faith in spiritual values | c:Their support for Western values | d:Their love of sports',
  'Correct': 'b'},
 {'MCQ': 'In which E

In [34]:
quiz=pd.DataFrame(quiz_table_data)
quiz

,MCQ,Choices,Correct
0,What was the main reason for Sayyid Qutb's cri...,a:Their support for the new Israeli state | b:...,d
1,In what year was Sayyid Qutb born?,a:1906 | b:1966 | c:1949 | d:1929,a
2,Where did Sayyid Qutb study the educational sy...,a:Stanford University | b:Colorado State Colle...,d
3,What was the title of Sayyid Qutb's first majo...,a:Ashwak (Thorns) | b:Muhimmat al-Sha’ir fi al...,c
4,What organization did Sayyid Qutb join in the ...,a:Muslim Brotherhood | b:Al-Qaeda | c:Islamic ...,a


In [35]:
quiz.to_csv("syedqutab.csv", index=False)

In [36]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'09_09_2024_16_22_11'